In [113]:
from textblob import TextBlob, Blobber
import tweepy
from os import environ,listdir
import json
import pandas as pd
from os.path import expanduser, join
from textblob_fr import PatternTagger, PatternAnalyzer
from textblob.sentiments import NaiveBayesAnalyzer

In [2]:
path = expanduser('~/spark_apps/unesco_mentions/')

In [3]:
df_data = []
for f in listdir(path):
    with open(join(path,f),'r') as fp:
        df_data.extend(json.load(fp))

In [7]:
mentions = pd.DataFrame(df_data)

In [27]:
path = expanduser('~/spark_apps/convo_roots/')
df_data = []
for f in listdir(path):
    with open(join(path,f),'r') as fp:
        df_data.extend(json.load(fp))

In [28]:
convos = pd.DataFrame(df_data)

In [15]:
convo_ids  = list(mentions['conversation_id'])

In [18]:
links = r'https?://[\w\.\d/]+'
hashtags = r'#[\w\d]+'
usernames = r'@[\w\d_]+'

In [29]:
convos = convos[(convos['lang']=='en') | (convos['lang']=='fr')]

In [23]:
mentions = mentions[(mentions['lang']=='en') | (mentions['lang']=='fr')]

In [87]:
mentions['text'] = (mentions['text'].str.replace(links,'', regex = True)
            .str.replace(hashtags,'', regex = True)
            .str.replace(usernames, '',regex = True)
            .str.replace(r'\s+',' ', regex = True)
            .str.replace(r'[^\w\s]','',regex=True)
            .str.lower())

In [88]:
convos['text'] = (convos['text'].str.replace(links,'', regex = True)
            .str.replace(hashtags,'', regex = True)
            .str.replace(usernames, '',regex = True)
            .str.replace(r'\s+',' ', regex = True)
            .str.replace(r'[^\w\s]','',regex=True)
            .str.lower())

In [33]:
from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

In [34]:
# wc_data = df.groupby('hashtags').sum(numeric_only=True)
# wc = wc_data.to_dict()['like_count']

# cloud = WordCloud(min_word_length=3, colormap = 'winter',max_words=20,width = 400, height = 200,prefer_horizontal = 0.95,
#         background_color = 'white',collocations = False).generate_from_frequencies(wc)

# fig, ax = plt.subplots()
# ax.axis('off')
# ax.imshow(cloud)

# return fig

In [89]:
mentions_en = mentions[mentions['lang']=='en']
mentions_fr = mentions[mentions['lang']=='fr']

In [90]:
mentions_en['text'].head()

0     it is literally a division what are you talki...
2     of all journalists killed in 2021 89 were men...
3     i see how stupid this tweet is but at the sam...
4     wtf does this even mean my god universities a...
6     and what about the 89 left hope you make twit...
Name: text, dtype: object

In [91]:
sentiment_en = pd.DataFrame([TextBlob(i).sentiment for i in list(mentions_en['text'])])

In [92]:
mentions_en = pd.concat((mentions_en.reset_index(),sentiment_en),axis = 1)

In [93]:
fr_blobber = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [94]:
sentiment_fr = pd.DataFrame([fr_blobber(i).sentiment for i in list(mentions_fr['text'])],columns = ['polarity','subjectivity'])

In [95]:
mentions_fr = pd.concat((mentions_fr.reset_index(),sentiment_fr),axis = 1)

In [96]:
ments = pd.concat((mentions_fr, mentions_en))


In [99]:
ments['sentiment'] = ''
ments['sentiment'].mask((ments['polarity'] <= 0.0001) & (ments['polarity'] >= -0.0001),'neutral',inplace=True)
ments['sentiment'].mask((ments['polarity'] < -0.5),'very negative',inplace=True)
ments['sentiment'].mask((ments['polarity'] < -0.0001) & (ments['polarity'] > -0.5),'negative',inplace=True)
ments['sentiment'].mask((ments['polarity'] > 0.5),'very positive',inplace=True)
ments['sentiment'].mask((ments['polarity'] > 0.0001) & (ments['polarity'] < 0.5),'positive',inplace=True)

In [117]:
list(ments[ments['sentiment']=='neutral']['text'])[-50:]

['join us tomorrow 9 nov at to discover the key contribution of to amp  9 am eet at ',
 ' we are concerned about the welfare of kabaka king ronald muwenda mutebi ii who stands for health amp wellbeing of all ugandans from  ',
 ' as if rest 89 male journalists were saying ',
 'today we are celebrating the launch of the   a joint initiative of       promoting a series of events operating around  1st online event starts today at 12 dont miss it',
 'prs s prireditelji podnebnega teka for the future of our kids the question of is basically a question of life and death and it is our responsibility to answer this challenges ',
 'today is intday to for crimes against journalists according to  9 out of 10 journalist killings go unpunished austria is partnering with amp to organise on 3  4 nov 2022 and promote accountability',
 ' shor temple',
 ' women are not being targeted tho they are dying cause they are in the middle of a warzone nobodys looking at journalists and only shooting female ones 

In [118]:
TextBlob('recognize and stop hazara genocide in afghanistan please be our voice', analyzer = NaiveBayesAnalyzer()).sentiment

Sentiment(classification='neg', p_pos=0.23313201149471613, p_neg=0.766867988505284)